# 0 摘要

本文主要是阿里MLP模型（Mixed Linear Model）的论文版，全称叫做Large Scale Piece-wise Linear Model。本文提出了该模型来解决CTR预估场景中高维稀疏、非线性拟合的建模问题，并提出一种高效地解L1和L2正则化的方式。在工程架构上，设计了一种求解该问题的分布式系统。MLP模型从2012年就开始在阿里进行了大规模的应用。

# 1 简介

CTR预估需要大量的数据进行建模学习，使得该问题变成了一个具有大规模数据和特征的建模问题。

传统的解决方案是使用分布式的逻辑回归进行建模，同时加入L1正则化可以产生稀疏解，缩短线上serving过程的时间。然而，逻辑回归本身是一种线性模型，对于CTR预估中的非线性特征无法很好的学习。

因此早期的解决思路主要有两个：
- 在特征工程上处理非线性信息，使得逻辑回归可以学习非线性特征，但人工成本是相当大的。
- 使用可以学习非线性关系的模型，例如FB提出的GBDT+LR，但是tree-based model不适合高维稀疏特征样本，因此后来又引入FM模型来进行向量化的学习，但FM对于二阶组合无法很好的学习

因此，本文提出了LS-PLM模型，采用分而治之的思想，将特征空间划分为多个local regions，然后在每个local regions拟合线性模型，最后将结果进行加权线性预测。这里，分片和加权求和是同时进行和joint learning的，并不是独立分隔开的。

LS-PLM主要在以下三个方面具有优势：
- Nonlinearity：通过划分足够多的regions，LS-PLM可以拟合任意复杂非线性函数
- Scalability：多机分布式训练与部署
- Sparsity：增加正则化，使得学习参数洗漱，加速serving

在训练方面，LS-PLM增加了L1正则化，使得优化目标变为non-convex和non-differential，本文基于directional derivatives和quasi-Newton法提出了一种高效的优化方法。

# 2 方法

## 2.1 Formulation

分而治之的思想（divide-and-conquer）：
- 划分特征空间
- 在各自特征空间建立线性模型

模型如下：

$$p(y=1|x)=\sum_{i=1}^m \sigma(u_i^T x)\eta(w_i^T x) = \sum_{i=1}^m \frac{exp(u_i^T x)}{\sum_{j=1}^m exp(u_j^T x)}\cdot \frac{1}{1+exp(-w_i^T x)}$$

其中，$\sigma(\cdot)$函数将特征空间划分为m个分片；$\eta(\cdot)$函数用来拟合该空间下的预测值。MLP可以被看做FOE model：$p(y=1|x)=\sum_{i=1}^m p(z=i|x)p(y|z=i, x)$

目标函数：

$$arg\min\Theta f(\Theta)=loss(\Theta)+\lambda ||\Theta||_{2,1}+\beta||\Theta||_1$$
$$loss(\Theta)=-\sum_{t=1}^n[y_t\log(p(y_t=1|x_t,\Theta))+(1-y_t)\log(p(y_t=0|x_t,\Theta))]$$

- L_{2,1}用来压缩参数scale
- L_1用来做特征筛选

# 3 Implementation

## 3.1 训练并行化

![](pictures/mlp-parallel.png)

## 3.2 Common Feature Trick

对于用户侧特征，由于对于很多ad都需要计算，因此直接提前计算好存为索引，需要的时候直接拿出来用就好